<a href="https://colab.research.google.com/github/adolfoguimaraes/inteligenciaartificial/blob/main/code/06_Prolog_Parte3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prolog com Python - Parte 3

Nesse notebook vamos explorar mais algunas conceitos de Prolog. 

As bases utilizadas neste documento estão disponíveis no repositório: https://github.com/adolfoguimaraes/inteligenciaartificial. 

Para facilitar o uso dessas bases aqui no Colab, essa base está replicada também no diretório https://drive.google.com/drive/folders/19euiVFFa32TmqR_WAE8Xer-8G273efXN?usp=sharing. 

In [1]:
# Para uso no colab
dataset_path = "" 

In [ ]:
# Para uso no github
dataset_path = "../datasets/kb_prolog/"

In [2]:
# Rodar essa célula apenas se tiver rodando no colab.

!sudo apt install swi-prolog
!pip install pyswip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swi-prolog is already the newest version (7.6.4+dfsg-2ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Carregar a biblioteca 
from pyswip import Prolog
from pyswip import Atom

# Instanciar o objeto
prolog = Prolog()

In [4]:
def atoms_to_strings(answer):
    if isinstance(answer, dict):
        result = {}
        for k in answer.keys():
            result[k] = atoms_to_strings(answer[k])
    elif isinstance(answer, list):
        result = [atoms_to_strings(val) for val in answer]
    elif isinstance(answer, Atom):
        result = answer.value
    elif isinstance(answer, int) or isinstance(answer, str):
        result = answer
    else:
        print("Unsupported result from Prolog: " + str(type(answer)) + str(answer))
    return result

In [5]:
## Método para consultar a base
def consultar_base(pergunta: str):

  """
    Esse método recebe uma pergunta com parâmetro e retorna True se assertiva 
    for verdadeira e False, caso contrário. Caso a pergunta seja com variáveis, 
    o método vai retornar uma lista de dicionários, onde a chave de cada 
    dicionário é uma variável passada. 

    :param pergunta: str
    
    :return : bool, list
  """

  result_ = prolog.query(pergunta)
  
  result_list = list(result_)

  if len(result_list) == 0: return False
  else:
    if len(result_list[0].keys()) == 0: return True
    else:
      return result_list


In [6]:
def consultar_base_com_lista(pergunta: str):
  result_ = prolog.query(pergunta)
  final_result = []
  for element in result_:
    list_ = atoms_to_strings(element)
    final_result.append(list_)

  return final_result

In [7]:
def atualizar_local(local: str):

  prolog.retract("here(X)")
  prolog.asserta("here("+local+")")

In [8]:
def inserir_fato(fato: str):
  prolog.asserta(fato)

## Problema 1 

Vamos trabalhar com a base de conhecimento da `base5.pl`.

In [9]:
# Carregar a base5.pl
prolog.consult(dataset_path + "base5.pl")

Essa base contém fatos sobre a possibilidade de pegar um trem direto entre duas cidades. No entanto, podemos viajar de um lugar para o outro trocando de trem. Sendo assim, escreva um predicado `travelFromTo/2` que diz se é
possível viajar de trem entre duas cidades. Por exemplo, quando perguntado:

`travelFromTo(nancy, staAvold)`

deve retornar True já que podemos seguir o caminho `nancy -> metz -> fahlquemont -> stAvold`.

_Atualize o arquivo base5.pl com a sua regra e teste nas linhas a seguir_

In [ ]:
# Carregue a base novamente e teste. 

## Problema 2 

Vamos considerar agora a base do arquivo `base6.pl`. 

In [10]:
# Carregar a base6.pl
prolog.consult(dataset_path + "base6.pl")

Escreva um predicado listatraducao(P, E) que traduz um lista de números em Português para a lista correspondente em Inglês. Por exemplo,

`listatraducao([um,nove,dois],X)`.

deve gerar:

`X = [one,nine,two].`

Seu programa deve funcionar também na direção contrária. Por exemplo, se você entra com a seguinte query:

`listatraducao(X,[one,seven,six,two]).`

deve ser retornado:

`X = [um,sete,seis,dois].`

_Atualize o arquivo base6.pl com a sua regra e teste nas linhas a seguir_

In [ ]:
# Carregue a base novamente e teste. 

## Problema 3 - Jogo da Nany

Para começar, vamos carregar o arquivo `base7.pl` para demonstrar alguns comandos novos para se trabalhar com Prolog. Essa base é apenas de exemplo. Ela indica o mapeamento de uma casa com os cômodos e a conexão entre eles. A regra `here` indica a posição de uma pessoa na casa. 



In [11]:
prolog.consult(dataset_path + "base7.pl")

In [12]:
consultar_base("door(cozinha, X)")

[{'X': 'escritorio'}]

In [13]:
consultar_base("door(cozinha, escritorio)")

True

A regra `here(X)` tem como caracaterística que ela é dinâmica. Ou seja, podemos altera-la durante a execuçã do programa. Vamos ver como funciona. 

In [14]:
# Primeiro vamos consultar onde o personagem está.
consultar_base("here(X)")

[{'X': 'cozinha'}]

Agora vamos utilizar o método `atualizar_local` que foi criado para alterar a posição na pessoa na casa. Esse método faz duas operações no prolog: `retract` e `asserta`. A primeira remove o conhecimento atual daquela regra e a segunda atribui a nova posição. 

In [16]:
atualizar_local("escritorio")

In [17]:
# Vamos consultar novamente a posição
consultar_base("here(X)")

[{'X': 'escritorio'}]

É possível também inserir novos fatos à base. Vamos inserir um novo fato à nossa base `has(escritorio, [desk, computer])`. Observer que esse fato não existe em nossa base. 

In [18]:
consultar_base_com_lista("has(escritorio, X)")

PrologError: ignored

Vamos inserir o fato na base. 

In [19]:
inserir_fato("has(escritorio, [desk, computer])")

In [20]:
consultar_base_com_lista("has(escritorio, X)")

[{'X': ['desk', 'computer']}]

Dada essa introdução, vamos fazer uma atividade. 

A proposta do laboratório é criar um jogo aventura. Nele, você é uma criança de 3 anos de idade que só consegue dormir com sua babá. Sendo assim, a sua missão é explorar a casa em busca da Babá que está escondida em algum lugar da casa. Você deve mapear os fatos e as regras desse problema para mapear um solução em prolog. Antes do mapeamento precisamos conhecer as seguintes informações:

* Os cômodos da casa e suas conexões;
* Os objetos e a localização de cada um deles;
* As propriedades destes objetos;
* Onde o jogador está no início do jogo;

Os cômodos e suas conexões podem ser vistos na imagem a seguir:

![](https://adolfoguimaraes.github.io/artificialintelligence/class5/imgs/502_casa_exemplo.png)

Com isso, podemos definir o predicado `room/1` com os fatos relacionados à imagem apresentada. Por exemplo, `room(office)`. é um fato extraídos das informações do mapa. Faça isso com os demais cômodos da casa.

Um outra informação que pode ser extraída do mapa é a representação das conexões por meio das portas. Sendo assim, crie o predicado `door/2` com os fatos que indicam onde existe um porta entre os cômodos.

Um outro predicado que precisa ser definido é o de `location/2` que possui dois argumentos. O primeiro é o objeto e o segundo sua localização. Crie todos os fatos correspondentes ao predicado `location/2` de acordo com a descrição a seguir:

| Local	| Objetos no local |
|---------| ---------------- |
| office |	desk, computer |
| kitchen	| apple, broccoli, crackers |
| desk |	flashlight. |
| cellar |	washing machine | 
| washing machine	| nani | 

Além das informações do mapa e da localização dos objetos. Existem outros fatos que precisam ser mapeados.

* **apple** e **crackers** são comestíveis (use o predicado `edible/1` para representar este fato)
* **broccoli** não é tão saboroso para criança (use o predicado `tastes_yucky/1` para representar este fato).

Por fim, defina o status inicial para o objeto flashlight a partir do predicado `turned_off/1` e a localização inicial da criança por meio do predicado `here/1`. No início do jogo, a criança está localizada na cozinha.

Uma vez que os fatos foram especificados faz-se necessário especificar uma série de regras essenciais para o jogo. São elas:

* **where_food/2**: dada uma comida, exibe onde ela está localizada.
* **connect/2**: indica se dois cômodos estão conectados. Um cômodo está conectado se existe uma porta entre eles.
* **list_things/2**: lista todos as coisas que estão em um determinado local. O primeiro parâmetro é o local e o segundo a lista de objetos no local.
* **list_connection/1**: dado um local indica todas as conexões que existem neste local. O primeiro parâmetro é o local e o segundo é uma lista com todas as conexões.
* **look/1**: dado um local, essa regra deve atualizar o fato here com a nova localização do jogador e escrever na tela o local onde ele está, os objetos que existem neste local e para quais locais ele pode ir a partir daquele local. Não esqueça que um objeto pode estar localizado em cima ou dentro de um outro objeto.

Até aqui vocês definiram o cenário do nosso jogo. Em seguida, deve permitir que a criança ande pela casa. Em cada cômodo que ela chegue, deve-se executar a regra look para que ela atualize sua localização e liste tudo que está vendo. Caso a babá (nani) não esteja no cômodo atual, o bebê deve escolher um outro cômodo e visitar este novo cômodo. Caso esteja, o jogo deve ser encerrado. O bebê só pode passar para um outro cômodo se exister uma conexão entre o cômodo atual e o novo cômodo.

A medida que a criança vai andando, o valor do predicado here/1 é atualizado pela regra look. Para isso vocês devem utilizar os comandos de atualização mostrados anteriormente no tutorial.

### Atividade

Implementar o jogo da Nani utilizando uma base de conhecimento em Prolog e a execução do programa a partir da linguagem Python. O jogo encerra quando o bebê encontrar a Nani. Enquanto ele não encontrar, o mesmo deve explorar cada cômodo informando o que tem no cômodo conforme descrição passada neste documento. 

Caso a criança pare em um cômado que tenha comida, ela vai comer desde de que seja uma comida que ela goste. Toda vez que a regra `look` for chamada deve também mostrar a lista de alimentos que a criança comeu.  

Vale ressaltar que conforme a base descrita, a Nani está escondida na máquina de lavar :)

Considere a seguinte base de conheciemento (arquivo: `base8.pl`): 

```prolog
:- dynamic here/1.

room(office).
room(hall).
room(kitchen).
room(diningroom).
room(cellar).

location(desk, office).
location(computer, office).
location(apple, kitchen).
location(broccoli, kitchen).
location(crackers, kitchen).
location(flashlight, desk).
location(washingmachine, cellar).
location(nani, washingmachine).

door(office, hall).
door(office, kitchen).
door(hall, diningroom).
door(diningroom, kitchen).
door(kitchen, cellar).

edible(apple).
edible(crackers).
tastes_yucky(broccoli).

turned_off(flashlight).
here(kitchen).
```

Crie regras que permitam fazer o que o jogo pede. Após a criação das regras, faça um programa em Python que consulte a base de conhecimento para permitir que a criança explore o ambiente. 

**Atenção:** Caso seu jogo receba uma base de conhecimento maior, com mais cômodos e mais situações. Ele deve funcionar do mesmo jeito. 

**Atenção 2:** Se um objeto está dentro ou em cima de um outro objeto que está em algum cômodo, esse primeiro objeto também está no cômodo. Por exemplo, se eu tenho uma caneta que está em cima de uma mesa dentro do escritório, a caneta também está no escritório. 



In [ ]:
prolog.consult("base8.pl")

In [ ]:
prolog.retract("conectado(X,Y)")

StopIteration: ignored

In [ ]:
inserir_fato("conectado(X, Y) :- door(X, Y); door(Y,X)")
inserir_fato("visitado(nenhum)")
inserir_fato("quem_posso_visitar(X, Y) :- conectado(X, Y), not(visitado(Y))")

In [ ]:
consultar_base("conectado(kitchen, X)")

[{'X': 'cellar'}, {'X': 'office'}, {'X': 'diningroom'}]

In [ ]:
consultar_base("quem_posso_visitar(kitchen, X)")

[{'X': 'cellar'}, {'X': 'office'}, {'X': 'diningroom'}]

In [ ]:
inserir_fato("visitado(cellar)")

In [ ]:
consultar_base("quem_posso_visitar(kitchen, X)")

[{'X': 'office'}, {'X': 'diningroom'}]

In [ ]:
local_atual = consultar_base('here(X)')[0]['X']
print("Eu estou na %s" % local_atual)
conexoes = consultar_base("conectado(" + local_atual + ", X)")
atualizar_local(conexoes[0]['X'])
local_atual = consultar_base('here(X)')[0]['X']
print("Eu estou na %s" % local_atual)

Eu estou na kitchen
Eu estou na cellar
